In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import os
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns
from tqdm import tqdm, tqdm_notebook
from pathlib import Path
# pd.set_option('display.max_columns', 1000)
# pd.set_option('display.max_rows', 400)
sns.set()

os.chdir('..')

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from project.ranker.ranker import RankingPredictor

In [3]:
%%time
rp = Pipeline([
    ('scale', StandardScaler()),
    ('estimator', RankingPredictor("ma_100", n_neighbors=15)),
])
df_mf, df_rank, df_scores, df_fold_scores = rp.named_steps['estimator'].get_data()

CPU times: user 54.9 s, sys: 3.67 s, total: 58.5 s
Wall time: 58.3 s


In [7]:
from sklearn.model_selection import train_test_split

X_train, _, y_train, _, y_scores_train, _ = train_test_split(df_mf.values,
                                                             df_rank.values,
                                                             df_scores.values,
                                                             test_size=0)
X_train.shape, y_train.shape

((60, 39), (60, 13))

In [191]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=2, random_state=42)
params = {'objective': 'lambdarank', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   2 | Trn_Spearman:  0.2977 | Val_Spearman:  0.2592 | Trn_ACCLoss:  0.0370 | Val_ACCLoss:  0.0682 | Trn_NDCG:  0.7352 | Val_NDCG:  0.7133
Fold   2 | #Est:   6 | Trn_Spearman:  0.3516 | Val_Spearman:  0.2408 | Trn_ACCLoss:  0.0152 | Val_ACCLoss:  0.0731 | Trn_NDCG:  0.7980 | Val_NDCG:  0.5778
Fold   3 | #Est:  24 | Trn_Spearman:  0.4100 | Val_Spearman:  0.2582 | Trn_ACCLoss:  0.0124 | Val_ACCLoss:  0.0797 | Trn_NDCG:  0.8507 | Val_NDCG:  0.5944
Fold   4 | #Est:   3 | Trn_Spearman:  0.3758 | Val_Spearman:  0.0321 | Trn_ACCLoss:  0.0202 | Val_ACCLoss:  0.0715 | Trn_NDCG:  0.7622 | Val_NDCG:  0.5557
Fold   5 | #Est:  23 | Trn_Spearman:  0.4199 | Val_Spearman:  0.0513 | Trn_ACCLoss:  0.0098 | Val_ACCLoss:  0.1202 | Trn_NDCG:  0.8485 | Val_NDCG:  0.5623
Fold   6 | #Est:  23 | Trn_Spearman:  0.4028 | Val_Spearman:  0.1877 | Trn_ACCLoss:  0.0153 | Val_ACCLoss:  0.0274 | Trn_NDCG:  0.8131 | Val_NDCG:  0.6443
Fold   7 | #Est:  98 | Trn_Spearman:  0.4237 | Val_Spearman: -0.0339 |

In [190]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=10, random_state=42)
params = {'objective': 'lambdarank', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   2 | Trn_Spearman:  0.2977 | Val_Spearman:  0.2592 | Trn_ACCLoss:  0.0370 | Val_ACCLoss:  0.0682 | Trn_NDCG:  0.7352 | Val_NDCG:  0.7133
Fold   2 | #Est:   6 | Trn_Spearman:  0.3516 | Val_Spearman:  0.2408 | Trn_ACCLoss:  0.0152 | Val_ACCLoss:  0.0731 | Trn_NDCG:  0.7980 | Val_NDCG:  0.5778
Fold   3 | #Est:  24 | Trn_Spearman:  0.4100 | Val_Spearman:  0.2582 | Trn_ACCLoss:  0.0124 | Val_ACCLoss:  0.0797 | Trn_NDCG:  0.8507 | Val_NDCG:  0.5944
Fold   4 | #Est:   3 | Trn_Spearman:  0.3758 | Val_Spearman:  0.0321 | Trn_ACCLoss:  0.0202 | Val_ACCLoss:  0.0715 | Trn_NDCG:  0.7622 | Val_NDCG:  0.5557
Fold   5 | #Est:  23 | Trn_Spearman:  0.4199 | Val_Spearman:  0.0513 | Trn_ACCLoss:  0.0098 | Val_ACCLoss:  0.1202 | Trn_NDCG:  0.8485 | Val_NDCG:  0.5623
Fold   6 | #Est:  23 | Trn_Spearman:  0.4028 | Val_Spearman:  0.1877 | Trn_ACCLoss:  0.0153 | Val_ACCLoss:  0.0274 | Trn_NDCG:  0.8131 | Val_NDCG:  0.6443
Fold   7 | #Est:  98 | Trn_Spearman:  0.4237 | Val_Spearman: -0.0339 |

In [68]:
from project.ranker.ltr_rankers import wide2long
X, y = wide2long(X_train, y_train)
X.shape, y.shape

((780, 40), (780,))

In [170]:
from scipy.stats import rankdata
y_pred = np.array([rankdata(models[0].predict(wide2long(x_[None,:], y_[None,:])[0]), 
                               method='ordinal') for x_, y_ in zip(X_train, y_train)])
y_pred.shape

(60, 13)

In [173]:
13 - y_train[0] + 1

array([12,  9,  8,  3,  1, 11, 10,  2, 13,  4,  6,  5,  7])

In [172]:
y_pred[0]

array([ 1,  2,  3,  4,  5,  8,  9, 10, 11, 12, 13,  6,  7])

In [174]:
from scipy.stats import spearmanr
spearmanr(13 - y_train[0] + 1, y_pred[0])

SpearmanrResult(correlation=-0.10989010989010989, pvalue=0.7208097995138618)

## Ranking, Regression, Classification

In [195]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=2, random_state=42)
params = {'objective': 'lambdarank', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   2 | Trn_Spearman:  0.2977 | Val_Spearman:  0.2592 | Trn_ACCLoss:  0.0370 | Val_ACCLoss:  0.0682 | Trn_NDCG:  0.7352 | Val_NDCG:  0.7133
Fold   2 | #Est:   6 | Trn_Spearman:  0.3516 | Val_Spearman:  0.2408 | Trn_ACCLoss:  0.0152 | Val_ACCLoss:  0.0731 | Trn_NDCG:  0.7980 | Val_NDCG:  0.5778
Fold   3 | #Est:  24 | Trn_Spearman:  0.4100 | Val_Spearman:  0.2582 | Trn_ACCLoss:  0.0124 | Val_ACCLoss:  0.0797 | Trn_NDCG:  0.8507 | Val_NDCG:  0.5944
Fold   4 | #Est:   3 | Trn_Spearman:  0.3758 | Val_Spearman:  0.0321 | Trn_ACCLoss:  0.0202 | Val_ACCLoss:  0.0715 | Trn_NDCG:  0.7622 | Val_NDCG:  0.5557
Fold   5 | #Est:  23 | Trn_Spearman:  0.4199 | Val_Spearman:  0.0513 | Trn_ACCLoss:  0.0098 | Val_ACCLoss:  0.1202 | Trn_NDCG:  0.8485 | Val_NDCG:  0.5623
Fold   6 | #Est:  23 | Trn_Spearman:  0.4028 | Val_Spearman:  0.1877 | Trn_ACCLoss:  0.0153 | Val_ACCLoss:  0.0274 | Trn_NDCG:  0.8131 | Val_NDCG:  0.6443
Fold   7 | #Est:  98 | Trn_Spearman:  0.4237 | Val_Spearman: -0.0339 |

In [194]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=2, random_state=42)
params = {'objective': 'regression', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   2 | Trn_Spearman:  0.4294 | Val_Spearman:  0.3901 | Trn_ACCLoss:  0.0621 | Val_ACCLoss:  0.1853 | Trn_NDCG:  0.6465 | Val_NDCG:  0.6706
Fold   2 | #Est:  72 | Trn_Spearman:  0.5380 | Val_Spearman:  0.2903 | Trn_ACCLoss:  0.0387 | Val_ACCLoss:  0.1764 | Trn_NDCG:  0.7101 | Val_NDCG:  0.6252
Fold   3 | #Est:  30 | Trn_Spearman:  0.4639 | Val_Spearman:  0.1951 | Trn_ACCLoss:  0.0622 | Val_ACCLoss:  0.1125 | Trn_NDCG:  0.6839 | Val_NDCG:  0.5400
Fold   4 | #Est:  32 | Trn_Spearman:  0.4380 | Val_Spearman:  0.2601 | Trn_ACCLoss:  0.0456 | Val_ACCLoss:  0.0480 | Trn_NDCG:  0.6749 | Val_NDCG:  0.7206
Fold   5 | #Est:  15 | Trn_Spearman:  0.4373 | Val_Spearman:  0.0595 | Trn_ACCLoss:  0.0644 | Val_ACCLoss:  0.1327 | Trn_NDCG:  0.6335 | Val_NDCG:  0.5640
Fold   6 | #Est:   1 | Trn_Spearman:  0.2000 | Val_Spearman: -0.0568 | Trn_ACCLoss:  0.1398 | Val_ACCLoss:  0.0802 | Trn_NDCG:  0.5635 | Val_NDCG:  0.5381
Fold   7 | #Est:   3 | Trn_Spearman:  0.4399 | Val_Spearman:  0.0330 |

In [196]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=2, random_state=42)
params = {'objective': 'binary', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   1 | Trn_Spearman: -0.0954 | Val_Spearman:  0.0375 | Trn_ACCLoss:  0.1136 | Val_ACCLoss:  0.1499 | Trn_NDCG:  0.5295 | Val_NDCG:  0.5097
Fold   2 | #Est:   1 | Trn_Spearman: -0.0900 | Val_Spearman: -0.0110 | Trn_ACCLoss:  0.1129 | Val_ACCLoss:  0.1561 | Trn_NDCG:  0.5317 | Val_NDCG:  0.4901
Fold   3 | #Est:   1 | Trn_Spearman: -0.0665 | Val_Spearman: -0.2225 | Trn_ACCLoss:  0.1216 | Val_ACCLoss:  0.0782 | Trn_NDCG:  0.5195 | Val_NDCG:  0.5999
Fold   4 | #Est:   1 | Trn_Spearman: -0.0980 | Val_Spearman:  0.0604 | Trn_ACCLoss:  0.1218 | Val_ACCLoss:  0.0761 | Trn_NDCG:  0.5231 | Val_NDCG:  0.5679
Fold   5 | #Est:   1 | Trn_Spearman: -0.0736 | Val_Spearman: -0.1593 | Trn_ACCLoss:  0.1107 | Val_ACCLoss:  0.1763 | Trn_NDCG:  0.5227 | Val_NDCG:  0.5713
Fold   6 | #Est:   1 | Trn_Spearman: -0.0648 | Val_Spearman: -0.2381 | Trn_ACCLoss:  0.1159 | Val_ACCLoss:  0.1295 | Trn_NDCG:  0.5292 | Val_NDCG:  0.5131
Fold   7 | #Est:   1 | Trn_Spearman: -0.1095 | Val_Spearman:  0.1639 |

## 10 runs - 10 folds

In [224]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=10, random_state=42)
params = {'objective': 'lambdarank', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   2 | Trn_Spearman:  0.2977 | Val_Spearman:  0.2592 | Trn_ACCLoss:  0.0370 | Val_ACCLoss:  0.0682 | Trn_NDCG:  0.7352 | Val_NDCG:  0.7133
Fold   2 | #Est:   6 | Trn_Spearman:  0.3516 | Val_Spearman:  0.2408 | Trn_ACCLoss:  0.0152 | Val_ACCLoss:  0.0731 | Trn_NDCG:  0.7980 | Val_NDCG:  0.5778
Fold   3 | #Est:  24 | Trn_Spearman:  0.4100 | Val_Spearman:  0.2582 | Trn_ACCLoss:  0.0124 | Val_ACCLoss:  0.0797 | Trn_NDCG:  0.8507 | Val_NDCG:  0.5944
Fold   4 | #Est:   3 | Trn_Spearman:  0.3758 | Val_Spearman:  0.0321 | Trn_ACCLoss:  0.0202 | Val_ACCLoss:  0.0715 | Trn_NDCG:  0.7622 | Val_NDCG:  0.5557
Fold   5 | #Est:  23 | Trn_Spearman:  0.4199 | Val_Spearman:  0.0513 | Trn_ACCLoss:  0.0098 | Val_ACCLoss:  0.1202 | Trn_NDCG:  0.8485 | Val_NDCG:  0.5623
Fold   6 | #Est:  23 | Trn_Spearman:  0.4028 | Val_Spearman:  0.1877 | Trn_ACCLoss:  0.0153 | Val_ACCLoss:  0.0274 | Trn_NDCG:  0.8131 | Val_NDCG:  0.6443
Fold   7 | #Est:  98 | Trn_Spearman:  0.4237 | Val_Spearman: -0.0339 |

In [225]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=10, random_state=42)
params = {'objective': 'regression', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   2 | Trn_Spearman:  0.4294 | Val_Spearman:  0.3901 | Trn_ACCLoss:  0.0621 | Val_ACCLoss:  0.1853 | Trn_NDCG:  0.6465 | Val_NDCG:  0.6706
Fold   2 | #Est:  72 | Trn_Spearman:  0.5380 | Val_Spearman:  0.2903 | Trn_ACCLoss:  0.0387 | Val_ACCLoss:  0.1764 | Trn_NDCG:  0.7101 | Val_NDCG:  0.6252
Fold   3 | #Est:  30 | Trn_Spearman:  0.4639 | Val_Spearman:  0.1951 | Trn_ACCLoss:  0.0622 | Val_ACCLoss:  0.1125 | Trn_NDCG:  0.6839 | Val_NDCG:  0.5400
Fold   4 | #Est:  32 | Trn_Spearman:  0.4380 | Val_Spearman:  0.2601 | Trn_ACCLoss:  0.0456 | Val_ACCLoss:  0.0480 | Trn_NDCG:  0.6749 | Val_NDCG:  0.7206
Fold   5 | #Est:  15 | Trn_Spearman:  0.4373 | Val_Spearman:  0.0595 | Trn_ACCLoss:  0.0644 | Val_ACCLoss:  0.1327 | Trn_NDCG:  0.6335 | Val_NDCG:  0.5640
Fold   6 | #Est:   1 | Trn_Spearman:  0.2000 | Val_Spearman: -0.0568 | Trn_ACCLoss:  0.1398 | Val_ACCLoss:  0.0802 | Trn_NDCG:  0.5635 | Val_NDCG:  0.5381
Fold   7 | #Est:   3 | Trn_Spearman:  0.4399 | Val_Spearman:  0.0330 |

In [226]:
%%time
import lightgbm
from project.ranker.ltr_rankers import cv_lgbm
from sklearn.model_selection import RepeatedKFold
kfolds = RepeatedKFold(10, n_repeats=10, random_state=42)
params = {'objective': 'binary', 
          'metric': 'ndcg', 
          'learning_rate': 1e-3,
#           'num_leaves': 50,
          'ndcg_at': y_train.shape[1],
          'min_data_in_leaf': 3,
          'min_sum_hessian_in_leaf': 1e-4}
results, models = cv_lgbm(lightgbm, X_train, y_train.shape[1] - y_train + 1, y_scores_train, kfolds, 
                  params, num_rounds=1000, early_stopping_rounds=50, verbose_eval=False)

Fold   1 | #Est:   1 | Trn_Spearman: -0.0954 | Val_Spearman:  0.0375 | Trn_ACCLoss:  0.1136 | Val_ACCLoss:  0.1499 | Trn_NDCG:  0.5295 | Val_NDCG:  0.5097
Fold   2 | #Est:   1 | Trn_Spearman: -0.0900 | Val_Spearman: -0.0110 | Trn_ACCLoss:  0.1129 | Val_ACCLoss:  0.1561 | Trn_NDCG:  0.5317 | Val_NDCG:  0.4901
Fold   3 | #Est:   1 | Trn_Spearman: -0.0665 | Val_Spearman: -0.2225 | Trn_ACCLoss:  0.1216 | Val_ACCLoss:  0.0782 | Trn_NDCG:  0.5195 | Val_NDCG:  0.5999
Fold   4 | #Est:   1 | Trn_Spearman: -0.0980 | Val_Spearman:  0.0604 | Trn_ACCLoss:  0.1218 | Val_ACCLoss:  0.0761 | Trn_NDCG:  0.5231 | Val_NDCG:  0.5679
Fold   5 | #Est:   1 | Trn_Spearman: -0.0736 | Val_Spearman: -0.1593 | Trn_ACCLoss:  0.1107 | Val_ACCLoss:  0.1763 | Trn_NDCG:  0.5227 | Val_NDCG:  0.5713
Fold   6 | #Est:   1 | Trn_Spearman: -0.0648 | Val_Spearman: -0.2381 | Trn_ACCLoss:  0.1159 | Val_ACCLoss:  0.1295 | Trn_NDCG:  0.5292 | Val_NDCG:  0.5131
Fold   7 | #Est:   1 | Trn_Spearman: -0.1095 | Val_Spearman:  0.1639 |

In [ ]:
# LambdaRank
Trn_Spearman:  0.3714 +/-0.0634 | Val_Spearman:  0.0922 +/-0.1403
Trn_ACCLoss:   0.0254 +/-0.0194 | Val_ACCLoss:   0.0921 +/-0.0564
Trn_NDCG:      0.7845 +/-0.0671 | Val_NDCG:      0.6157 +/-0.0555

# Regression        
Trn_Spearman:  0.3531 +/-0.1215 | Val_Spearman:  0.1699 +/-0.1319
Trn_ACCLoss:   0.0847 +/-0.0324 | Val_ACCLoss:   0.1166 +/-0.0723
Trn_NDCG:      0.6294 +/-0.0510 | Val_NDCG:      0.5891 +/-0.0665
        
# Binary Classification        
Trn_Spearman: -0.0821 +/-0.0170 | Val_Spearman: -0.0821 +/-0.1533
Trn_ACCLoss:   0.1172 +/-0.0064 | Val_ACCLoss:   0.1172 +/-0.0573
Trn_NDCG:      0.5275 +/-0.0054 | Val_NDCG:      0.5275 +/-0.0486   